* **GP**-Games Played    **MIN**-Minutes Played    **FG%**-Field Goal Percentage    **FGM**-Field Goals Made
* **FGA**-Field Goals Attempted    **3P%**-3 Point Field Goals Percentage    **3PM**-3 Point Field Goals Made
* **3PA**-3 Point Field Goals Attempted    **FT%**-Free Throw Percentage    **FTM**-Free Throws Made
* **FTA**-Free Throws Attempted **REB**-Rebounds **OREB**-Offensive Rebounds **DREB**-Defensive Rebounds
* **AST**-Assists **STL**-Steals **BLK**-Blocks **TOV**-Turnovers **PF**-Personal Fouls **PTS**-Points
* **PER**-efficiency value **WS**-Win Shares

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#load dataset
df = pd.read_csv("04-12_NBA_rookies_data.csv", header=None)

In [2]:
#load dataset
df1 = pd.read_csv("04-12_NBA_rookies_data_without3PP.csv", header=None, encoding='utf-8')
#df1 = df1.astype(float)
ds = df1.values
x = ds[:,0:23]

#  PCA analysis

In [4]:
def zeroMean(dataMat):        
    meanVal=np.mean(dataMat,axis=0)     #get mean by columns
    newData=dataMat-meanVal  
    return newData,meanVal


def percentage2n(eigVals,percentage):  
    sortArray=np.sort(eigVals)          #ascend order 
    sortArray=sortArray[-1::-1]           
    arraySum=sum(sortArray)  
    tmpSum=0  
    num=0  
    for i in sortArray:  
        tmpSum+=i  
        num+=1  
        if tmpSum>=arraySum*percentage:  
            return num  


def pca(dataMat,percentage=0.99):  
    newData,meanVal=zeroMean(dataMat)  
    covMat=np.cov(newData,rowvar=0)      #covariance  
    eigVals,eigVects=np.linalg.eig(np.mat(covMat))      
    n=percentage2n(eigVals,percentage)                 #need n dimensionol data to get the convariance percentage
    eigValIndice=np.argsort(eigVals)            #ascend order 
    n_eigValIndice=eigValIndice[-1:-(n+1):-1]   
    n_eigVect=eigVects[:,n_eigValIndice]        
    lowDDataMat=newData*n_eigVect               #lower dimensional data  
    reconMat=(lowDDataMat*n_eigVect.T)+meanVal  #reconstruct data  
    return lowDDataMat,reconMat,n 

# PCA processing

In [5]:
lowDDataMat,reconMat,n = pca(x)
print('auto choose '+str(n)+' dimentional')
lowDDataMat

auto choose 5 dimentional


matrix([[ 66.98303368, -32.80212741, -15.43392667,   7.5355039 ,
          -2.23328347],
        [ 48.94320987,  -5.03255543, -11.6712219 ,   1.10170476,
           4.22327956],
        [ 29.28591998,  18.11223535,  -5.3553855 ,   3.80480391,
           0.69330913],
        ...,
        [-52.32370277, -14.23029569,  -2.44296531,  -6.16752451,
           5.0028104 ],
        [-46.42545777,  -8.67923521,  -3.30853984, -11.48551298,
           7.75618201],
        [-49.96009269, -11.45113396,  -4.59694688,  -6.95903878,
           3.82868821]])